In [4]:
from __future__ import print_function, unicode_literals, absolute_import, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import glob
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from csbdeep.utils import _raise, Path, axes_check_and_normalize,axes_dict, move_image_axes, move_channel_for_backend, backend_channels_last
from externalmodels.unet import *
import tensorflow as tf
from tifffile import imread
from csbdeep.utils import Path, download_and_extract_zip_file, plot_some
from csbdeep.io import save_tiff_imagej_compatible
from csbdeep.models import CARE

In [5]:
basedir = '/data/u934/service_imagerie/v_kapoor/StarDistData/TimeData/'
basedirResults = '/data/u934/service_imagerie/v_kapoor/StarDistData/JuliaResultsData/Unet/'



BaseDirModels = 'models/UNet_model.h5'

print(BaseDirModels)

models/UNet_model.h5


In [6]:
Path = os.path.join(basedir, '*.tif')
X = []
Y = []
Names = []

filesRaw = glob.glob(Path)
maxtime = 0
for fname in filesRaw:
     x = imread(fname)
     X.append(x)
     Names.append(fname)
     if X[0].shape[0] > maxtime:
      maxtime =  X[0].shape[0]  
X.sort
Names.sort
SegmentedLabels = [];
labels = np.array(X[0])
print(labels.shape)
for i in range(len(X)):
    y = np.zeros((maxtime, X[0].shape[1], X[0].shape[2]))
    image = X[i]
    
    y[:X[i].shape[0],:,:] = image[:,:,:]
    Y.append(y) 
Y.sort
axes = 'TYX'
print('Total number of images to be segmented = ', len(X))
print('Image size = ', Y[0].shape)
print('Time dim = ', maxtime )
timerange = maxtime 
count = min(0, len(X))
print(timerange)

(241, 321, 314)
Total number of images to be segmented =  1
Image size =  (241, 321, 314)
Time dim =  241
241


In [7]:

model = keras.models.load_model(BaseDirModels, custom_objects={"_func":my_binary_crossentropy((1,10)), \
                                                                  "acc1":acc1, "acc0":acc0})

In [8]:
def ApplyModelovertime(modelName,i):
      
    img = Y[i]
    
    for j in range(timerange):
      
      print('Time point = ' , j, 'Image number = ', i)
      smallimg = img[j,:,:]
      smallimg = np.expand_dims(smallimg, axis = 0)
      smallimg = np.expand_dims(smallimg, axis = 3)  
      segmented = model.predict(smallimg)
      output = segmented[0,:,:,0]
      labels [j,:,:] = output[:,:] 
      print(labels.shape, output.shape)  
      
        
        
        
    SegmentedLabels.append(labels)
        
        


In [9]:

for i in range(0, len(Y)):
    ApplyModelovertime(model ,i)
     
    print(len(SegmentedLabels))

     

Time point =  0 Image number =  0


InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [1,16,160,156] vs. shape[1] = [1,16,160,157]
	 [[Node: concatenate_1/concat = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](concatenate_1/concat-0-TransposeNHWCToNCHW-LayoutOptimizer, activation_4/Relu, concatenate_1/concat-2-LayoutOptimizer)]]

In [ ]:

print(len(SegmentedLabels))
for i in range(0, len(SegmentedLabels)):
           print('Saving file'  + basedirResults  + os.path.basename(Names[i]))
           save_tiff_imagej_compatible((basedirResults + os.path.basename(Names[i])) , SegmentedLabels[i], axes)   
           print(SegmentedLabels[i].shape,len(SegmentedLabels) )  

In [ ]:

LastRaw = Y[len(SegmentedLabels) - 1]

smallimg = LastRaw[timerange - 2,:,:]
Lastimage= SegmentedLabels[len(SegmentedLabels) - 1]
smallSegmentedLabels = Lastimage[timerange - 2,:,:]
plt.figure(figsize = (16, 10))
plt.subplot(111); plt.imshow(smallimg, cmap = 'gray'); plt.axis('off'); plt.title('Raw Image')
plt.figure(figsize = (16, 10))
plt.subplot(121); plt.imshow(smallSegmentedLabels , cmap = 'gray'); plt.axis('off'); plt.title('Segmentation (sec last timepoint of last image)')
